In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange

from flygym.mujoco.arena import BaseArena, FlatTerrain, GappedTerrain, BlocksTerrain, MixedTerrain
from flygym.mujoco.examples.obstacle_arena import ObstacleOdorArena

import flygym.common
import flygym.mujoco
import flygym.mujoco.preprogrammed
from flygym.mujoco.examples.turning_controller import HybridTurningNMF

In [2]:
run_time = 1
sim_params = flygym.mujoco.Parameters(
    timestep=1e-4, render_mode="saved", render_playspeed=0.2, draw_contacts=True
)
actuated_joints = flygym.mujoco.preprogrammed.all_leg_dofs

In [3]:
data_path = flygym.common.get_data_path("flygym", "data")
with open(data_path / "behavior" / "210902_pr_fly1.pkl", "rb") as f:
    data = pickle.load(f)

In [4]:
target_num_steps = int(run_time / sim_params.timestep)
data_block = np.zeros((len(actuated_joints), target_num_steps))
input_t = np.arange(len(data["joint_LFCoxa"])) * data["meta"]["timestep"]
output_t = np.arange(target_num_steps) * sim_params.timestep
for i, dof in enumerate(actuated_joints):
    data_block[i, :] = np.interp(output_t, input_t, data[dof])

In [4]:
# sim_params = Parameters(
#     timestep=1e-4,
#     render_mode="saved",
#     render_playspeed=0.5,
#     render_window_size=(800, 608),
#     enable_olfaction=True,
#     enable_adhesion=True,
#     draw_adhesion=False,
#     render_camera="birdeye_cam",
# )

from flygym.mujoco.arena import OdorArena

# We start by creating a simple arena
flat_terrain_arena = FlatTerrain()

# Then, we add visual and olfactory features on top of it
arena = ObstacleOdorArena(
    terrain=flat_terrain_arena,
    obstacle_positions=np.array([(7.5, 0), (12.5, 5), (17.5, -5)]),
    marker_size=0.5,
    obstacle_colors=[(0.14, 0.14, 0.2, 1), (0.2, 0.8, 0.2, 1), (0.2, 0.2, 0.8, 1)],
    user_camera_settings=((13, -18, 9), (np.deg2rad(65), 0, 0), 45),
)
contact_sensor_placements = [
    f"{leg}{segment}"
    for leg in ["LF", "LM", "LH", "RF", "RM", "RH"]
    for segment in ["Tibia", "Tarsus1", "Tarsus2", "Tarsus3", "Tarsus4", "Tarsus5"]
]

nmf = HybridTurningNMF(
    sim_params=sim_params,
    arena = arena,
    spawn_pos=(0, 0, 0.2),
    contact_sensor_placements=contact_sensor_placements,
)


In [7]:
data_path = flygym.common.get_data_path("flygym", "data")
with open(data_path / "behavior" / "210902_pr_fly1.pkl", "rb") as f:
    data = pickle.load(f)

In [8]:
target_num_steps = int(run_time / sim_params.timestep)
data_block = np.zeros((len(actuated_joints), target_num_steps))
input_t = np.arange(len(data["joint_LFCoxa"])) * data["meta"]["timestep"]
output_t = np.arange(target_num_steps) * sim_params.timestep
for i, dof in enumerate(actuated_joints):
    data_block[i, :] = np.interp(output_t, input_t, data[dof])

In [13]:
magnitude_hist = []
obs, info = nmf.reset(seed=0)
for i in trange(int(run_time / nmf.sim_params.timestep)):
    curr_time = i * nmf.sim_params.timestep
    action = np.array([1.2, 0.2])
    obs, reward, terminated, truncated, info = nmf.step(action)
    nmf.render()
    magnitude_hist.append(nmf.cpg_network.curr_magnitudes.copy())

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:45<00:00, 219.79it/s]


In [20]:
from IPython.display import Video
nmf.save_video("./outputs/kinematic_replay.mp4")
Video("./outputs/kinematic_replay.mp4")

ValueError: To embed videos, you must pass embed=True (this may make your notebook files huge)
Consider passing Video(url='...')

In [11]:
from scipy.signal import find_peaks

# get the joint angle of the right hind leg tibia
rh_tibia_angle = data_block[actuated_joints.index("joint_RHTibia")]

# ===============================================================================================
# TODO: detect peaks in the tibia angle of the right hind leg using scipy.signal.find_peaks
# see https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html for usage
peak_frame_indices = find_peaks(rh_tibia_angle)[0]
# ===============================================================================================

cycle_n_frames = round(np.diff(peak_frame_indices).mean())

In [12]:
from scipy.interpolate import interp1d

data_block_cycle = np.zeros((len(actuated_joints), cycle_n_frames))

for a, b in np.lib.stride_tricks.sliding_window_view(peak_frame_indices, 2):
    old_indices = np.arange(b - a)
    new_indices = np.linspace(0, old_indices.max(), cycle_n_frames)
    data_block_cycle += interp1d(old_indices, data_block[:, a:b])(new_indices)

data_block_cycle /= len(peak_frame_indices) - 1

In [21]:
# nmf = flygym.mujoco.NeuroMechFly(
#     sim_params=sim_params,
#     init_pose="stretch",
#     actuated_joints=actuated_joints,
#     control="position",
# )
obs, info = nmf.reset()
for i in trange(target_num_steps):
    # =========================================================================
    # TODO: replay the locomotor kinematics at 2× speed by skipping frames
    # you will find the data_block_cycle and cycle_n_frames variables useful
    joint_pos = data_block_cycle[:, i * 2 % cycle_n_frames]
    # =========================================================================
    action = {"joints": joint_pos}
    obs, reward, terminated, truncated, info = nmf.step(action)
    nmf.render()

nmf.save_video("./outputs/kinematic_replay_fast.mp4")

  0%|          | 0/10000 [00:00<?, ?it/s]


TypeError: unhashable type: 'slice'

In [13]:
import numpy as np

# Create a 2D numpy array
data_block = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print("Original array:")
print(data_block)

# Reverse the order of elements in the second dimension
data_block_backwards = data_block[:, ::-1]
print("\nArray after reversing the order of elements in the second dimension:")
print(data_block_backwards)

Original array:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

Array after reversing the order of elements in the second dimension:
[[3 2 1]
 [6 5 4]
 [9 8 7]]


In [14]:
data_block_backwards = data_block_cycle.copy()

# =============================================================================
# TODO: modify the data_block_backwards to make the fly walk backwards
data_block_backwards = data_block_backwards[:, ::-1]  # reverse the order of the frames
# =============================================================================

# =============================================================================
# Alternatively, replay the kinematics of the hind legs in the front legs and
# vice versa
for i, (dof, angles) in enumerate(zip(actuated_joints, data_block_cycle)):
    # reassign the joint angles
    # (LF -> RH, RF -> LH, LM -> RM, RM -> LM, LH -> RF, RH -> LF)
    orig_dof = dof.translate(str.maketrans("LRFH", "RLHF"))[:8] + dof[8:]
    orig_angles = data_block_cycle[actuated_joints.index(orig_dof)]
    # rescale to match the range of the joint angles of that leg
    data_block_backwards[i] = (
        orig_angles - orig_angles.min()
    ) / orig_angles.ptp() * angles.ptp() + angles.min()

In [15]:

obs, info = nmf.reset()
for i in trange(target_num_steps):
    joint_pos = data_block_backwards[:, i % cycle_n_frames]
    action = {"joints": joint_pos}
    obs, reward, terminated, truncated, info = nmf.step(action)
    nmf.render()

nmf.save_video("./outputs/kinematic_replay_backwards.mp4")

  0%|          | 0/10000 [00:00<?, ?it/s]

TypeError: unhashable type: 'slice'

In [16]:
from typing import List, Tuple

In [17]:
class Turning_NMF(HybridTurningNMF):
    def __init__(self, *args, **kwargs):
        self,
        actuated_joints: List = flygym.mujoco.preprogrammed.all_leg_dofs,


In [39]:
class MyClass:
    def __init__(self, arg1, arg2):
        self.arg1 = arg1
        self.arg2 = arg2*2

    def display_args(self):
        print(f"arg1: {self.arg1}, arg2: {self.arg2}")

In [40]:
class inti(MyClass):
    def __init__(self, arg2, arg3):
        self.arg2 = arg2
        self.arg3 = arg3
        super().__init__( arg2)

    def display_args(self):
        print(f"arg1: {self.arg1}, arg2: {self.arg2}, arg3: {self.arg3}")   

In [41]:
class right(inti):
    def __init__(self, arg1, arg2, arg3, arg4):
        self.arg4 = arg4
        super().__init__(arg1, arg2, arg3)
    
    def display_args(self):
        print(f"arg1: {self.arg1}, arg2: {self.arg2}, arg3: {self.arg3}, arg4: {self.arg4}")

In [35]:
# Create an instance of inti
inti_instance = inti("Hello", "World", "Python")

# Call the display_args method
inti_instance.display_args()  # Output: arg1: Hello, arg2: World, arg3: Python

arg1: Hello, arg2: WorldWorld, arg3: Python


In [ ]:
def Turning_NMF_backwards(HybridTurningNMF):

    def __init__(self, *args, **kwargs):
        
    
        super().__init__(*args, **kwargs)
        self.actuated_joints = flygym.mujoco.preprogrammed.all_leg_dofs